In [3]:
import polars as pl

In [4]:
res = pl.read_parquet("/Users/borja/Documents/Somniumrema/projects/genai/grag/pipeline_outcomes/df_results_rerank.parquet")


# Increment 'question_number' by 1
res = res.with_columns(
    (pl.col("question_number") - 1).alias("question_number")
)

# Display the updated DataFrame
res.head(1)

question_number,question,answer,retrieval_result,context
i64,str,str,list[str],str
1,"""When did Airbnb go public, wha…","""According to the context, Airb…","[""The text discusses the founding story of Airbnb, focusing on the friendship and collaboration between Joe and Brian, who met at RISD. Joe sends Brian a CritBun, symbolizing their decision to start a company together. Brian moves to San Francisco to join Joe and their roommate Nate, a programmer. The trio faces financial challenges and decides to leverage the high demand for accommodations during the World Design Congress in San Francisco to start a business. This event leads to the inception of Airbnb, which revolutionizes the concept of peer-to-peer lodging. The text also touches on the growth and impact of Airbnb, including its IPO and global reach."", ""The text describes the early days of Airbnb, focusing on how the founders, Joe and Brian, came up with the idea and their initial struggles to get the business off the ground. They started by offering air mattresses in their apartment during a design conference in San Francisco, which led to the creation of AirBed and Breakfast. The founders faced challenges in raising funds and initially struggled to attract users. They eventually found success by leveraging events like South by Southwest and the presidential conventions to gain bookings. The text also mentions their involvement with Y Combinator and their creative marketing tactics, such as creating Obama O's and Cap'n McCain's cereals."", … ""The text discusses various aspects of Airbnb's business model, including the ease of switching for consumers, different types of marketplaces, revenue generation, and profitability. It also touches on the value created for hosts and guests, the impact on housing supply and prices, and challenges such as host consistency and quality issues. The text raises concerns about host churn and the potential negative effects of Airbnb's business model on housing markets.""]","""Similarly for gifts, it's a li…"


In [5]:
res = res.rename({
    "question_number": "numero_pregunta",
    "question": "pregunta",
    "answer": "respuesta",
})

In [6]:
# Filter out rows where 'response' contains "I don't know" (case-insensitive)
filtered_df = res.filter(
    pl.col("respuesta").str.contains("I don't know", literal=False) |
    pl.col("respuesta").str.contains("I don't have enough", literal=False) |
    pl.col("respuesta").str.contains("I don't see any ", literal=False) |
    pl.col("respuesta").str.contains("I don't have the information", literal=False) 
)

# Display the filtered DataFrame
print("\nFiltered DataFrame:")
filtered_df


Filtered DataFrame:


numero_pregunta,pregunta,respuesta,retrieval_result,context
i64,str,str,list[str],str
15,"""What boots and jacket we often…","""I apologize, but I don't see a…","[""The text discusses the story of Amazon and its founder, Jeff Bezos. It mentions the importance of focusing on what you can excel at, surrounding yourself with people you admire, and the impact of working with those you love. The text also touches on the history of Amazon, Jeff Bezos' background, and the influence of his family on his life. Additionally, it highlights the launch of an Acquired merch store, an interview with Austin Federa from the Solana Foundation, and the Fundrise Innovation Fund. The text provides insights into the growth and success of Amazon, as well as the significance of key individuals and events in its history."", ""The text discusses the early life and career of Jeff Bezos, focusing on his family background, education, and early work experiences. It highlights key moments such as his adoption by Mike Bezos, his upbringing in Houston during the Apollo era, and his education at Princeton University. The text also delves into Bezos' early career, including his work at startups and in finance, leading up to his pivotal role at D. E. Shaw & Co., a quant hedge fund. The narrative emphasizes Bezos' entrepreneurial spirit, his connection to technology and finance, and his rapid rise within the industry."", … ""The text discusses Amazon's history, including its failures and successes, comparing it to other successful companies like SpaceX and Tesla. It highlights Amazon's ability to learn from failures and make bold investment decisions. The text also touches on Jeff Bezos' approach to customer focus and competitor focus, as well as Amazon's survival during the dot-com crash. The text concludes with a discussion on grading Amazon's performance up to 2007, giving it an A+ for its execution and survival through challenging times. Additionally, the text includes recommendations for the Rick Rubin episode on the Lex Friedman podcast, books by Ursula Le Guin, the album ""My Beautiful Dark Twisted Fantasy"" by Kanye West, and the video game Elden Ring.""]","""I think a lot of times, people…"


In [33]:
quality = pl.read_csv("/Users/borja/Documents/Somniumrema/projects/genai/grag/pipeline_outcomes/quality_results_vrerank2.csv")

quality = quality.rename({
    "row_number": "numero_pregunta",
    "question": "pregunta",
    "answer": "respuesta",
    "coherence_score": "coherencia",
    "answer_relevancy_score": "relevancia",
})

In [54]:
quality.filter(pl.col("coherencia") < 0.14 )

numero_pregunta,pregunta,respuesta,coherencia,faithfulness_score,relevancia
i64,str,str,f64,f64,f64
9,"""What is American Crown Jewel?""","""The exact information about ""A…",0.139735,1.0,1.0
28,"""can you tell me total number o…","""The exact number of people wor…",0.139167,1.0,1.0


In [58]:
import plotly.graph_objects as go

# Calculate mean coherence for correct and incorrect answers
correct_mean = quality.filter(
    ~pl.col("numero_pregunta").is_in([14, 15, 44])
)["relevancia"].mean()

# Filter out rows where 'response' contains "I don't know" (case-insensitive)
incorrect_mean = quality.filter(
    pl.col("numero_pregunta").is_in([14, 15, 44])
)["relevancia"].mean()

# Create figure
fig = go.Figure()

# Add bars for mean coherence
fig.add_trace(
    go.Bar(
        x=['CORRECT', 'INCORRECT'],
        y=[correct_mean, incorrect_mean],
        marker_color=['green', 'red']
    )
)



# Update layout
fig.update_layout(
    width=1200,  # Adjust width as needed
    height=600,  # Adjust height as needed
    title_text="Media de Relevancia por Respuesta",
    yaxis_title="Relevancia",
    xaxis_title="Respuesta",
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=0, r=0, t=30, b=0),
    showlegend=False
)

# Remove gridlines
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False, zeroline=True)

# Show the plot
fig.show()

In [8]:
results_1 = pl.read_parquet("/Users/borja/Documents/Somniumrema/projects/genai/grag/pipeline_outcomes/df_results_sistema5.parquet")

In [11]:
results_1

results_1 = results_1.rename({
    "question_number": "numero_pregunta",
    "question": "pregunta",
    "answer": "respuesta",
    "query_time": "tiempo_computación"
})

In [14]:
results_1

numero_pregunta,pregunta,respuesta,retrieval_result,context,tiempo_computación
i64,str,str,list[str],str,f64
2,"""When did Airbnb go public, wha…","""Airbnb went public on December…","[""The text discusses the founding story of Airbnb, focusing on the friendship and collaboration between Joe and Brian, who met at RISD. Joe sends Brian a CritBun, symbolizing their decision to start a company together. Brian moves to San Francisco to join Joe and their roommate Nate, a programmer. The trio faces financial challenges and decides to leverage the high demand for accommodations during the World Design Congress in San Francisco to start a business. This event leads to the inception of Airbnb, which revolutionizes the concept of peer-to-peer lodging. The text also touches on the growth and impact of Airbnb, including its IPO and global reach."", ""The text describes the early days of Airbnb, focusing on how the founders, Joe and Brian, came up with the idea and their initial struggles to get the business off the ground. They started by offering air mattresses in their apartment during a design conference in San Francisco, which led to the creation of AirBed and Breakfast. The founders faced challenges in raising funds and initially struggled to attract users. They eventually found success by leveraging events like South by Southwest and the presidential conventions to gain bookings. The text also mentions their involvement with Y Combinator and their creative marketing tactics, such as creating Obama O's and Cap'n McCain's cereals."", … ""The text discusses various aspects of Airbnb's business model, including the ease of switching for consumers, different types of marketplaces, revenue generation, and profitability. It also touches on the value created for hosts and guests, the impact on housing supply and prices, and challenges such as host consistency and quality issues. The text raises concerns about host churn and the potential negative effects of Airbnb's business model on housing markets.""]","""Similarly for gifts, it's a li…",6.588006
3,"""Why did Wimdu unlike Airbnb no…","""Wimdu, a competitor to Airbnb,…","[""The text discusses the founding story of Airbnb, focusing on the friendship and collaboration between Joe and Brian, who met at RISD. Joe sends Brian a CritBun, symbolizing their decision to start a company together. Brian moves to San Francisco to join Joe and their roommate Nate, a programmer. The trio faces financial challenges and decides to leverage the high demand for accommodations during the World Design Congress in San Francisco to start a business. This event leads to the inception of Airbnb, which revolutionizes the concept of peer-to-peer lodging. The text also touches on the growth and impact of Airbnb, including its IPO and global reach."", ""The text describes the early days of Airbnb, focusing on how the founders, Joe and Brian, came up with the idea and their initial struggles to get the business off the ground. They started by offering air mattresses in their apartment during a design conference in San Francisco, which led to the creation of AirBed and Breakfast. The founders faced challenges in raising funds and initially struggled to attract users. They eventually found success by leveraging events like South by Southwest and the presidential conventions to gain bookings. The text also mentions their involvement with Y Combinator and their creative marketing tactics, such as creating Obama O's and Cap'n McCain's cereals."", … ""The text discusses various aspects of Airbnb's business model, including the ease of switching for consumers, different types of marketplaces, revenue generation, and profitability. It also touches on the value created for hosts and guests, the impact on housing supply and prices, and challenges such as host consistency and quality issues. The text raises concerns about host churn and the potential negative effects of Airbnb's business model on housing markets.""]","""Similar

In [19]:
# Filter out rows where 'response' contains "I don't know" (case-insensitive)
filtered_df = results_1.filter(
    pl.col("respuesta").str.contains("I don't know", literal=False) |
    pl.col("respuesta").str.contains("I don't have enough", literal=False) |
    pl.col("respuesta").str.contains("I don't see any ", literal=False) |
    pl.col("respuesta").str.contains("I don't have the information", literal=False) 
)

# Display the filtered DataFrame
print("\nFiltered DataFrame:")
filtered_df


Filtered DataFrame:


numero_pregunta,pregunta,respuesta,retrieval_result,context,tiempo_computación
i64,str,str,list[str],str,f64


In [29]:
filtered_time = results_1.filter(pl.col("tiempo_computación")>13)

filtered_time

numero_pregunta,pregunta,respuesta,retrieval_result,context,tiempo_computación
i64,str,str,list[str],str,f64
15,"""What was the final words by Am…","""In a Quora post in 2011, Amazo…","[""The text discusses the origins and growth of Amazon Web Services (AWS), the cloud computing pioneer. It mentions that AWS is a significant and influential business, even more so than Amazon's retail arm. The text explores various origin stories of AWS, including the misconception that it was born out of excess capacity during Amazon's seasonal retail spikes. It also touches on the challenges faced by Amazon in the early 2000s and the pivotal role of Andy Jassy, the current CEO of Amazon, in the development of AWS. The text highlights the importance of AWS in powering the internet and its impact on modern technology."", ""The text discusses the evolution of Amazon Web Services (AWS) and its origins. It highlights how AWS was not initially about selling excess capacity but rather a deliberate strategy to provide APIs and IT infrastructure services to developers. The text also delves into the concept of Web 2.0, APIs, and the early days of AWS within Amazon Associates. The narrative emphasizes the intentional strategy behind AWS's creation and its fundamental innovation in the IT industry."", … ""The text discusses various topics, including a conversation about Reed Hastings, the CEO and founder of Netflix, and his background in engineering. It also delves into Amazon's strategy of launching and maintaining services, particularly in AWS, and the challenges of generating cash flow to justify its market cap. The text also touches on the Marvel show Moon Knight and an interview with John Carmack by Lex Fridman. The interview covers Carmack's legendary career in technology, his work in VR and AI, and his relationship with John Romero. The text concludes with information about the Acquired Slack community, merchandise, and the acquired LP episodes.""]","""Anyway, that's not how I choos…",15.058366
38,"""what is one decision Porsche m…","""One significant decision Porsc…","[""The text discusses various topics, including investment strategies, market conditions, long-term investing, the importance of people and culture in businesses, and the history of Porsche. It mentions the significance of emotional stability in investing, the impact of tough times on capital deployment, and the value of investing in businesses with strong people and culture. Additionally, it delves into the history of Porsche, highlighting Ferdinand Porsche's role in the German auto industry, his collaboration with Mercedes-Benz, and the founding of the Dr. Ing. h.c. Ferdinand Porsche Consulting and Design Services for Motor Vehicles company, which later became the company that produces Porsches. Ferdinand Porsche's connections to the Nazis and the involvement of other individuals like Anton Piech and Adolf Rosenberger are also mentioned."", ""The text discusses the intertwined histories of Volkswagen and Porsche, highlighting their origins during World War II and their post-war developments. It mentions how Ferdinand Porsche's stake in Porsche was appropriated by the Nazis, leading to the creation of Volkswagen and the design of the Volkswagen Beetle. After the war, Volkswagen was revitalized by British officer Major Ivan Hirst, who saw the potential in the Beetle and convinced the British command to restart production. Meanwhile, Porsche faced challenges post-war, with Ferdinand and his son Ferry being arrested for war crimes. Ferry Porsche later started a new company in Austria, focusing on fixing military vehicles and eventually developing the first Porsche prototype, emphasizing the importance of lightweight design and powerful engines in creating a more enjoyable driving experience."", … ""The text discusses various factors contributing to Porsche's success in the automotive industry, such as scale economies, deep partnerships with other car brands, heritage, brand reputation

In [30]:
import plotly.graph_objects as go

# Calculate mean value
mean_tiempo_computacion = results_1['tiempo_computación'].mean()
filtered_ids = filtered_df["numero_pregunta"].to_list()
# Create figure
fig = go.Figure()

# Add bars for tiempo_computación
fig.add_trace(
    go.Bar(
        x=results_1['numero_pregunta'],
        y=results_1['tiempo_computación'],
        name='Tiempo de Computación',
        marker_color=['red' if (row['numero_pregunta'] == 54 )
                      else 'green' for row in results_1.iter_rows(named=True)],
        legendgroup='Answer',
        showlegend=False
    )
)

# Add mean line
fig.add_hline(y=mean_tiempo_computacion, line_dash="dash", line_color="blue")

# Add annotation for mean line
fig.add_annotation(
    x=results_1['numero_pregunta'].max(),
    y=mean_tiempo_computacion,
    text=f"Tiempo de Computación medio: {mean_tiempo_computacion:.2f} s",
    showarrow=False,
    yshift=10,
    xanchor='right'
)

# Add legend items
fig.add_trace(go.Bar(x=[None], y=[None], name='CORRECT', marker_color='green', legendgroup='Answer'))
fig.add_trace(go.Bar(x=[None], y=[None], name='INCORRECT', marker_color='red', legendgroup='Answer'))

# Set axis titles
fig.update_xaxes(title_text="Número de Pregunta")
fig.update_yaxes(title_text="Tiempo de Computación (s)")

# Update layout for transparency and no frame
fig.update_layout(
    width=1200,  # Adjust width as needed
    height=600,  # Adjust height as needed
    title_text="Coste por Respuesta (segundos)",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        traceorder='reversed'
    ),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=50, r=50, t=100, b=50),
    yaxis=dict(zeroline=True, range=[0, max(7, results_1['tiempo_computación'].max() * 1.1)])
)

# Remove gridlines
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

# Show the plot
fig.show()
